In [1]:
import sqlite3

# Check SQLite version
print(sqlite3.sqlite_version)

3.45.3


# Load the Insurance Claims Dataset
We will load the dataset into a pandas DataFrame before storing it in SQLite.

In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv("../data/insurance_claims.csv")

# Show first 5 rows to verify
df.head()


,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,10/17/2014,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,6/27/2006,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,9/6/2000,OH,100/300,2000,1413.14,5000000,430632,...,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,5/25/1990,IL,250/500,2000,1415.74,6000000,608117,...,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,6/6/2014,IL,500/1000,1000,1583.91,6000000,610706,...,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


# Create SQLite Database and Store Data
Now that the dataset is loaded, we will store it in an SQLite database.


In [3]:
import sqlite3

# Connect to SQLite (creates a database in memory)
conn = sqlite3.connect(":memory:")

# Store the dataset as a SQL table
df.to_sql("insurance_claims", conn, index=False, if_exists="replace")

# Verify if the table was created
query = "SELECT name FROM sqlite_master WHERE type='table';"
pd.read_sql(query, conn)


,name
0,insurance_claims


# View Column Names in SQLite
This query will display all column names in the "insurance_claims" table.


In [4]:
query = "PRAGMA table_info(insurance_claims);"
pd.read_sql(query, conn)


,cid,name,type,notnull,dflt_value,pk
0,0,months_as_customer,INTEGER,0,None,0
1,1,age,INTEGER,0,None,0
2,2,policy_number,INTEGER,0,None,0
3,3,policy_bind_date,TEXT,0,None,0
4,4,policy_state,TEXT,0,None,0
5,5,policy_csl,TEXT,0,None,0
6,6,policy_deductable,INTEGER,0,None,0
7,7,policy_annual_premium,REAL,0,None,0
8,8,umbrella_limit,INTEGER,0,None,0
9,9,insured_zip,INTEGER,0,None,0


# Count Total Claims and Fraud Cases
This query counts the total number of claims and identifies how many were fraudulent.


In [5]:
query = """
SELECT 
    COUNT(*) AS total_claims,
    SUM(CASE WHEN fraud_reported = 'Y' THEN 1 ELSE 0 END) AS total_fraudulent_claims,
    (SUM(CASE WHEN fraud_reported = 'Y' THEN 1 ELSE 0 END) * 100.0) / COUNT(*) AS fraud_percentage
FROM insurance_claims;
"""
pd.read_sql(query, conn)


,total_claims,total_fraudulent_claims,fraud_percentage
0,1000,247,24.7


# Identify High-Risk Incident Types
This query finds which types of incidents are more likely to be fraudulent.


In [6]:
query = """
SELECT 
    incident_type,
    COUNT(*) AS total_incidents,
    SUM(CASE WHEN fraud_reported = 'Y' THEN 1 ELSE 0 END) AS fraudulent_cases,
    (SUM(CASE WHEN fraud_reported = 'Y' THEN 1 ELSE 0 END) * 100.0) / COUNT(*) AS fraud_rate
FROM insurance_claims
GROUP BY incident_type
ORDER BY fraud_rate DESC;
"""
pd.read_sql(query, conn)


,incident_type,total_incidents,fraudulent_cases,fraud_rate
0,Single Vehicle Collision,403,117,29.032258
1,Multi-vehicle Collision,419,114,27.207637
2,Parked Car,84,8,9.523810
3,Vehicle Theft,94,8,8.510638


# Find Suspiciously High Claim Amounts
This query retrieves claims with unusually high total claim amounts.


In [7]:
query = """
SELECT 
    total_claim_amount,
    fraud_reported
FROM insurance_claims
ORDER BY total_claim_amount DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,total_claim_amount,fraud_reported
0,114920,N
1,112320,Y
2,108480,N
3,108030,N
4,107900,Y
5,105820,N
6,105040,N
7,104610,Y
8,103560,N
9,101860,N


# Detect Fraud Trends Based on Vehicle Make
This query determines which car brands are associated with the highest fraud rates.


In [8]:
query = """
SELECT 
    auto_make,
    COUNT(*) AS total_claims,
    SUM(CASE WHEN fraud_reported = 'Y' THEN 1 ELSE 0 END) AS fraudulent_cases,
    (SUM(CASE WHEN fraud_reported = 'Y' THEN 1 ELSE 0 END) * 100.0) / COUNT(*) AS fraud_rate
FROM insurance_claims
GROUP BY auto_make
ORDER BY fraud_rate DESC
LIMIT 10;
"""
pd.read_sql(query, conn)


,auto_make,total_claims,fraudulent_cases,fraud_rate
0,Mercedes,65,22,33.846154
1,Ford,72,22,30.555556
2,Audi,69,21,30.434783
3,Volkswagen,68,19,27.941176
4,BMW,72,20,27.777778
5,Chevrolet,76,21,27.631579
6,Honda,55,14,25.454545
7,Dodge,80,20,25.000000
8,Suburu,80,19,23.750000
9,Saab,80,18,22.500000


# Check If Deductibles Affect Fraud
This query examines the relationship between policy deductibles and fraud rates.


In [9]:
query = """
SELECT 
    policy_deductable,  -- ✅ Use the correct column name
    COUNT(*) AS total_claims,
    SUM(CASE WHEN fraud_reported = 'Y' THEN 1 ELSE 0 END) AS fraudulent_cases,
    (SUM(CASE WHEN fraud_reported = 'Y' THEN 1 ELSE 0 END) * 100.0) / COUNT(*) AS fraud_rate
FROM insurance_claims
GROUP BY policy_deductable
ORDER BY policy_deductable DESC;
"""
pd.read_sql(query, conn)


,policy_deductable,total_claims,fraudulent_cases,fraud_rate
0,2000,307,81,26.384365
1,1000,351,79,22.507123
2,500,342,87,25.438596
